In [1]:
import pandas as pd

df = pd.read_csv("../Data/0_example.txt", header=None)
df.head()

,0
0,4
1,L 3 animals fear war
2,P 2 smile woman
3,P 2 woman pearl
4,L 3 fear raft survivors


In [2]:
df = df.drop(0)
df.reset_index(drop=True, inplace=True) 
df.head()

,0
0,L 3 animals fear war
1,P 2 smile woman
2,P 2 woman pearl
3,L 3 fear raft survivors


In [3]:
def transform_data(row):
    parts = row.split()
    return {
        "type": parts[0],
        "tagNumber": int(parts[1]),
        "tags": parts[2:]
    }

df_transformed = df[0].apply(transform_data)
df_transformed = pd.DataFrame(df_transformed.tolist())
df_transformed.head()

,type,tagNumber,tags
0,L,3,"[animals, fear, war]"
1,P,2,"[smile, woman]"
2,P,2,"[woman, pearl]"
3,L,3,"[fear, raft, survivors]"


In [4]:
df_transformed.reset_index(inplace=True)
df_transformed.head()

,index,type,tagNumber,tags
0,0,L,3,"[animals, fear, war]"
1,1,P,2,"[smile, woman]"
2,2,P,2,"[woman, pearl]"
3,3,L,3,"[fear, raft, survivors]"
